# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 62 new papers today
          23 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/22 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2210.13472


extracting tarball to tmp_2210.13472...

 done.
Retrieving document from  https://arxiv.org/e-print/2210.13483


extracting tarball to tmp_2210.13483... done.
Retrieving document from  https://arxiv.org/e-print/2210.13489


...

extracting tarball to tmp_2210.14123...

 done.
Retrieving document from  https://arxiv.org/e-print/2210.14125


/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2210.14123/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'clumps' from 'tmp_2210.14123/clumps.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'individual' from 'tmp_2210.14123/individual.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_deck_2/

extracting tarball to tmp_2210.14125...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.13511-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.13511) | **SDSS-IV MaStar: Stellar Parameter Determination with  Continuum-Supplemented Full-Spectrum Fitting**  |
|| Daniel Lazarz, et al. -- incl., <mark>Lewis Hill</mark> |
|*Appeared on*| *2022-10-26*|
|*Comments*| *22 pages, 21 figures. A&A in press*|
|**Abstract**| We present a stellar parameter catalog built to accompany the MaStar Stellar Library, which is a comprehensive collection of empirical, medium-resolution stellar spectra. We constructed this parameter catalog by using a multicomponent $\chi^{2}$ fitting approach to match MaStar spectra to models generated by interpolating the ATLAS9-based BOSZ model spectra. The total $\chi^{2}$ for a given model is defined as the sum of components constructed to characterize narrow-band features of observed spectra (e.g., absorption lines) and the broadband continuum shape separately. Extinction and systematics due to flux calibration were taken into account in the fitting. The $\chi^{2}$ distribution for a given region of model space was sampled using a Markov Chain Monte Carlo (MCMC) algorithm, the data from which were then used to extract atmospheric parameter estimates ($T_{eff}$, log g, [Fe/H], and [$\alpha$/Fe]), their corresponding uncertainties, and direct extinction measurements. Two methods were used to extract parameters and uncertainties: one that accepts the MCMC's prescribed minimum-$\chi^{2}$ result, and one that uses Bayesian inference to compute a likelihood-weighted mean from the $\chi^{2}$ distribution sampled by the MCMC. Results were evaluated for internal consistency using repeat observations where available and by comparing them with external data sets (e.g., APOGEE-2 and Gaia DR2). Our spectral-fitting exercise reveals possible deficiencies in current theoretical model spectra, illustrating the potential power of MaStar spectra for helping to improve the models. This paper represents an update to the parameters that were originally presented with SDSS-IV DR17. The MaStar parameter catalog containing our BestFit results is available on the SDSS-IV DR17 website as part of version 2 of the MaStar stellar parameter value-added catalog. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.14076-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.14076) | **A long life of excess: The interacting transient SN 2017hcc**  |
|| S. Moran, et al. -- incl., <mark>P.J. Brown</mark> |
|*Appeared on*| *2022-10-26*|
|*Comments*| *Accepted to A&A*|
|**Abstract**| In this study we present the results of a five-year follow-up campaign of the long-lived type IIn supernova SN 2017hcc, found in a spiral dwarf host of near-solar metallicity. The long rise time (57 $\pm$ 2 days, ATLAS $o$ band) and high luminosity (peaking at $-$20.78 $\pm$ 0.01 mag in the ATLAS $o$ band) point towards an interaction of massive ejecta with massive and dense circumstellar material (CSM). The evolution of SN 2017hcc is slow, both spectroscopically and photometrically, reminiscent of the long-lived type IIn, SN 2010jl. An infrared (IR) excess was apparent soon after the peak, and blueshifts were noticeable in the Balmer lines starting from a few hundred days, but appeared to be fading by around +1200 days. We posit that an IR light echo from pre-existing dust dominates at early times, with some possible condensation of new dust grains occurring at epochs >$\sim$+800 days. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.13472-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.13472) | **Asymmetrical tidal tails of open star clusters: stars crossing their  cluster's prah challenge Newtonian gravitation**  |
|| Pavel Kroupa, et al. -- incl., <mark>Henri M.J. Boffin</mark> |
|*Appeared on*| *2022-10-26*|
|*Comments*| *28 pages, 19 figures, MNRAS, published*|
|**Abstract**| After their birth a significant fraction of all stars pass through the tidal threshold (prah) of their cluster of origin into the classical tidal tails. The asymmetry between the number of stars in the leading and trailing tails tests gravitational theory. All five open clusters with tail data (Hyades, Praesepe, Coma Berenices, COIN-Gaia 13, NGC 752) have visibly more stars within dcl = 50 pc of their centre in their leading than their trailing tail. Using the Jerabkova-compact-convergent-point (CCP) method, the extended tails have been mapped out for four nearby 600-2000 Myr old open clusters to dcl>50 pc. These are on near-circular Galactocentric orbits, a formula for estimating the orbital eccentricity of an open cluster being derived. Applying the Phantom of Ramses code to this problem, in Newtonian gravitation the tails are near-symmetrical. In Milgromian dynamics (MOND) the asymmetry reaches the observed values for 50 < dcl/pc < 200, being maximal near peri-galacticon, and can slightly invert near apo-galacticon, and the K\"upper epicyclic overdensities are asymmetrically spaced. Clusters on circular orbits develop orbital eccentricity due to the asymmetrical spill-out, therewith spinning up opposite to their orbital angular momentum. This positive dynamical feedback suggests Milgromian open clusters to demise rapidly as their orbital eccentricity keeps increasing. Future work is necessary to better delineate the tidal tails around open clusters of different ages and to develop a Milgromian direct n-body code. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.14089-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.14089) | **Transverse oscillations in 3D along Ca II K bright fibrils in the Solar  chromosphere**  |
|| Sepideh Kianfar, et al. -- incl., <mark>Jorrit Leenaarts</mark> |
|*Appeared on*| *2022-10-26*|
|*Comments*| *12 pages, 12 figures*|
|**Abstract**| Fibrils in the solar chromosphere carry transverse oscillations as determined from non-spectroscopic imaging data. They are estimated to carry an energy flux of several kW m$^{-2}$, which is a significant fraction of the average chromospheric radiative energy losses. We aim to determine oscillation properties of fibrils not only in the plane-of-the-sky (horizontal) direction, but also along the line-of-sight (vertical) direction. We obtained imaging-spectroscopy data in Fe I 6173, Ca II 8542, and Ca II K obtained with the Swedish 1-m Solar Telescope. We created a sample of 120 bright Ca II K fibrils and measured their horizontal motions. Their vertical motion was determined through non-LTE inversion of the observed spectra. We determined the periods and velocity amplitudes of the fibril oscillations, as well as phase differences between vertical and horizontal oscillations in the fibrils. The bright Ca II K fibrils carry transverse waves with a mean period of $2.2\times10^2$ s, and a horizontal velocity amplitude of 2 km s$^{-1}$, consistent with earlier results. The mean vertical velocity amplitude is 1 km s$^{-1}$. We find that 118 out of the 120 fibrils carry waves in both the vertical and horizontal directions, and 55 of those have identical periods. For those 55, we find that all phase differences between $0$ and $2\pi$ occur, with a mild but significant preference for linearly polarized waves (phase difference of $0$ or $\pi$). The results are consistent with the scenario where transverse waves are excited by granular buffeting at the photospheric footpoints of the fibrils. Estimates of transverse wave flux based on imaging data only are too low because they ignore the contribution of the vertical velocity. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.14116-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.14116) | **Model constraints based on the IceCube neutrino non-detection of GRB  201009A**  |
|| Shunke Ai, <mark>He Gao</mark> |
|*Appeared on*| *2022-10-26*|
|*Comments*| *6 pages, 2 figures*|
|**Abstract**| GRB 221009A is a bright Gamma-ray burst (GRB) with isotropic energy being larger than $10^{54} ~{\rm ergs}$. Its fairly low redshift makes it a promising candidate for high energy neutrino detection. However, a neutrino search for this GRB reported by the \emph{IceCube} collaboration yielded a null result. In this paper, we utilize the upper limit from IceCube observation to test different GRB prompt emission models. We find that, at least for this specific burst, the dissipative photosphere model could be ruled out in a large parameter space. The internal shock model can survive only with a large bulk motion Lorentz factor $\Gamma$, where the most stringent and conservative constraints are $\Gamma > \sim 400$ and $\Gamma > \sim 200$, respectively. For the Internal-collision-induced Magnetic Reconnection and Turbulence (ICMART) model, the constraint from GRB 221009A is modest. Under ICMART model, only for extreme situations when most dissipated energy deposit into protons and all accelerated protons are suitable for producing neutrinos, a slightly large bulk motion ($\Gamma > \sim 400$) is required. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.14123-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.14123) | **JWST reveals a possible $z \sim 11$ galaxy merger in triply-lensed  MACS0647$-$JD**  |
|| Tiger Yu-Yang Hsiao, et al. -- incl., <mark>Lily Whitler</mark>, <mark>Lillian Santos-Olmsted</mark>, <mark>Alaina Henry</mark>, <mark>Lisa Kewley</mark> |
|*Appeared on*| *2022-10-26*|
|*Comments*| *27 pages, 14 figures, submitted to Nature*|
|**Abstract**| MACS0647$-$JD is a triply-lensed $z\sim11$ galaxy originally discovered with the Hubble Space Telescope. Here we report new JWST imaging, which clearly resolves MACS0647$-$JD as having two components that are either merging galaxies or stellar complexes within a single galaxy. Both are very small, with stellar masses $\sim10^8\,M_\odot$ and radii $r<100\,\rm pc$. The brighter larger component "A" is intrinsically very blue ($\beta\sim-2.6$), likely due to very recent star formation and no dust, and is spatially extended with an effective radius $\sim70\,\rm pc$. The smaller component "B" appears redder ($\beta\sim-2$), likely because it is older ($100-200\,\rm Myr$) with mild dust extinction ($A_V\sim0.1\,\rm mag$), and a smaller radius $\sim20\,\rm pc$. We identify galaxies with similar colors in a high-redshift simulation, finding their star formation histories to be out of phase. With an estimated stellar mass ratio of roughly 2:1 and physical projected separation $\sim400\,\rm pc$, we may be witnessing a galaxy merger 400 million years after the Big Bang. We also identify a candidate companion galaxy C $\sim3\,{\rm kpc}$ away, likely destined to merge with galaxies A and B. The combined light from galaxies A+B is magnified by factors of $\sim$8, 5, and 2 in three lensed images JD1, 2, and 3 with F356W fluxes $\sim322$, $203$, $86\,\rm nJy$ (AB mag 25.1, 25.6, 26.6). MACS0647$-$JD is significantly brighter than other galaxies recently discovered at similar redshifts with JWST. Without magnification, it would have AB mag 27.3 ($M_{UV}=-20.4$). With a high confidence level, we obtain a photometric redshift of $z=10.6\pm0.3$ based on photometry measured in 6 NIRCam filters spanning $1-5\rm\mu m$, out to $4300\,\r{A}$ rest-frame. JWST NIRSpec observations planned for January 2023 will deliver a spectroscopic redshift and a more detailed study of the physical properties of MACS0647$-$JD. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.14125-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.14125) | **Rossiter-McLaughlin detection of the 9-month period transiting exoplanet  HIP41378 d**  |
|| S. Grouffal, et al. -- incl., <mark>J. Lillo-Box</mark> |
|*Appeared on*| *2022-10-26*|
|*Comments*| *Accepted for publication in A&A*|
|**Abstract**| The Rossiter-McLaughlin (RM) effect is a method that allows us to measure the orbital obliquity of planets, which is an important constraint that has been used to understand the formation and migration mechanisms of planets, especially for hot Jupiters. In this paper, we present the RM observation of the Neptune-sized long-period transiting planet HIP41378 d. Those observations were obtained using the HARPS-N/TNG and ESPRESSO/ESO-VLT spectrographs over two transit events in 2019 and 2022. The analysis of the data with both the classical RM and the RM Revolutions methods allows us to confirm that the orbital period of this planet is 278 days and that the planet is on a prograde orbit with an obliquity of $\lambda$ = 57.1+26.4-17.9 degrees, a value which is consistent between both methods. HIP41378 d is the longest period planet for which the obliquity was measured so far. We do not detect transit timing variations with a precision of 30 and 100 minutes for the 2019 and 2022 transits, respectively. This result also illustrates that the RM effect provides a solution to follow-up from the ground the transit of small and long-period planets such as those that will be detected by the forthcoming ESA's PLATO mission. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.13866-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.13866) | **Design, integration, and test of the scientific payloads on-board the  HERMES constellation and the SpIRIT mission**  |
|| Y. Evangelista, et al. -- incl., <mark>P. Hedderman</mark> |
|*Appeared on*| *2022-10-26*|
|*Comments*| *arXiv admin note: substantial text overlap with arXiv:2101.03032*|
|**Abstract**| HERMES (High Energy Rapid Modular Ensemble of Satellites) is a space-borne mission based on a constellation of nano-satellites flying in a low-Earth orbit (LEO). The six 3U CubeSat buses host new miniaturized instruments hosting a hybrid Silicon Drift Detector/GAGG:Ce scintillator photodetector system sensitive to X-rays and gamma-rays. HERMES will probe the temporal emission of bright high-energy transients such as Gamma-Ray Bursts (GRBs), ensuring a fast transient localization (with arcmin-level accuracy) in a field of view of several steradians exploiting the triangulation technique. With a foreseen launch date in late 2023, HERMES transient monitoring represents a keystone capability to complement the next generation of gravitational wave experiments. Moreover, the HERMES constellation will operate in conjunction with the Space Industry Responsive Intelligent Thermal (SpIRIT) 6U CubeSat, to be launched in early 2023. SpIRIT is an Australian-Italian mission for high-energy astrophysics that will carry in a Sun-synchronous orbit (SSO) an actively cooled HERMES detector system payload. On behalf of the HERMES collaboration, in this paper we will illustrate the HERMES and SpIRIT payload design, integration and tests, highlighting the technical solutions adopted to allow a wide-energy-band and sensitive X-ray and gamma-ray detector to be accommodated in a 1U Cubesat volume. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

73  publications in the last 7 days.
	 _build/html/2210.12412.md
	 _build/html/2210.11926.md
	 _build/html/2210.11551.md
	 _build/html/2210.11103.md
	 _build/html/2210.10707.md
	 _build/html/2210.09930.md
	 _build/html/2210.08996.md
	 _build/html/2210.08517.md
	 _build/html/2210.08510.md
	 _build/html/2210.08078.md
	 _build/html/2210.06504.md
	 _build/html/2210.05695.md
	 _build/html/2210.05595.md
	 _build/html/2210.05505.md
	 _build/html/2210.04721.md
	 _build/html/2210.03615.md
	 _build/html/2210.01811.md
	 _build/html/2210.01809.md
	 _build/html/2210.01556.md
	 _build/html/2210.00863.md
	 _build/html/2210.00560.md
	 _build/html/2209.15403.md
	 _build/html/2209.15110.md
	 _build/html/2209.14410.md
	 _build/html/2209.14396.md
	 _build/html/2209.13717.md
	 _build/html/2209.13336.md
	 _build/html/2209.12906.md
	 _build/html/2209.11781.md
	 _build/html/2209.11602.md
	 _build/html/2209.11248.md
	 _build/html/2209.11096.md
	 _build/html/2209.10618.md
	 _build/html/2209.09914.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers